In [1]:
#import
import numpy as np
import pandas as pd
import scipy.special as spcl

In [2]:
class ann:
    def __init__(self, input_nodes, hidden_nodes, output_node, learningrate):
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_node

        # ! calculate all hidden weights
        self.whh = []
        for i in range(0, len(self.hnodes)):
            # check if multilayer or not
            if i == 0:
                self.whh.append(
                    np.random.normal(
                        0.0, pow(self.inodes, -0.5), (self.inodes, self.hnodes[0])
                    )
                )
            else:
                self.whh.append(
                    np.random.normal(
                        0.0,
                        pow(self.hnodes[i - 1], -0.5),
                        (self.hnodes[i - 1], self.hnodes[i]),
                    )
                )

        # ! calculate hidden to output
        self.who = np.random.normal(
            0.0, pow(self.hnodes[-1], -0.5), (self.hnodes[-1], self.onodes)
        )

        self.lr = learningrate

        self.activation_function = lambda x: spcl.expit(x)
        self.deactivation_function = lambda x: x * (1.0 - x)

    def train(self, input_list, target_list):
        inputs = np.array(input_list, ndmin=2)
        targets = np.array(target_list, ndmin=2)

        # * feed forward
        # ? calculate all hidden input output
        hidden_inputs = []
        hidden_outputs = []
        for i in range(len(self.whh)):
            if i == 0:
                hidden_inputs.append(np.dot(inputs, self.whh[i]))
                hidden_outputs.append(self.activation_function(hidden_inputs[-1]))
            else:
                hidden_inputs.append(np.dot(hidden_outputs[i - 1], self.whh[i]))
                hidden_outputs.append(self.activation_function(hidden_inputs[-1]))

        # ? claculate hidden to output
        final_inputs = np.dot(hidden_outputs[-1], self.who)
        final_outputs = self.activation_function(final_inputs)

        # * error calculation
        # ?error out output
        output_error = targets - final_outputs

        # ? error of hidden
        hidden_error = []
        for i in range(1, len(self.whh) + 1):
            if i == 1:
                hidden_error.append(np.dot(output_error, self.who.T))
            else:
                hidden_error.append(np.dot(hidden_error[-1], self.whh[-(i - 1)].T))

        hidden_error = hidden_error[::-1]

        # * error reduction
        # ? reduce error of all hidden
        for i in range(len(self.whh)):
            if i == 0:
                self.whh[i] += inputs.T @ (
                    self.lr
                    * hidden_error[i]
                    * self.deactivation_function(hidden_outputs[i])
                )
            else:
                self.whh[i] += hidden_outputs[i - 1].T @ (
                    self.lr
                    * hidden_error[i]
                    * self.deactivation_function(hidden_outputs[i])
                )

        # ? reduce error of ouput
        self.who += hidden_outputs[-1].T @ (
            self.lr * output_error * self.deactivation_function(final_outputs)
        )

    def query(self, inputs):
        inputs = np.array(inputs, ndmin=2)

        hidden_inputs = []
        hidden_outputs = []
        for i in range(len(self.whh)):
            if i == 0:
                hidden_inputs.append(np.dot(inputs, self.whh[i]))
                hidden_outputs.append(self.activation_function(hidden_inputs[-1]))
            else:
                hidden_inputs.append(np.dot(hidden_outputs[i - 1], self.whh[i]))
                hidden_outputs.append(self.activation_function(hidden_inputs[-1]))

        final_inputs = np.dot(hidden_outputs[-1], self.who)
        final_outputs = self.activation_function(final_inputs)

        return final_outputs



In [3]:
# load dataset
dataset = pd.read_csv('phone.csv')

In [4]:
dataset = {
    "x" : dataset.drop(labels=['Class'],axis=1),
    "y": dataset["Class"] - 1
}

In [5]:
a = ann(5, [2], 1, 0.5)
for i in range(0, 1000):
    a.train(np.array(dataset["x"]),np.array(dataset["y"]).reshape([-1,1]))
    break
a.query(np.array(dataset["x"]))

array([[2.44115736e-01],
       [6.04464568e-08],
       [1.52007281e-07],
       ...,
       [1.52007281e-07],
       [4.96203745e-01],
       [1.52007281e-07]])